In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
# preprocess docs. Chunk playbook.md into individual md-files. Chunk right before ##-headings. Name the files after the heading.
import re
import os

def chunk_md_file(file_path):
    with open(file_path) as f:
        content = f.read()
    chunks = re.split(r'(?m)^## ', content)
    for chunk in chunks[1:]:
        heading = chunk.split('\n')[0]
        with open(f'./knowledge/{heading}.md', 'w') as f:
            f.write(f'## {chunk}')


def chunk_md_files():
    for file in os.listdir('./docs'):
        if file.endswith('.md'):
            chunk_md_file(f'./docs/{file}')

chunk_md_files()

In [3]:
# read the content of the knowledge folder and create a list of dictionaries with the following structure:
# {'title': '...', 'content': '...'}
# Use the title of the md-file as the title and the content of the md-file as the content
def read_knowledge_folder():
    knowledge = []
    for file in os.listdir('./knowledge'):
        if file.endswith('.md'):
            with open(f'./knowledge/{file}') as f:
                content = f.read()
            knowledge.append({'title': file[:-3], 'content': content})
    return knowledge

knowledge = read_knowledge_folder()
len(knowledge)>0

True

To build our knowledge base we need _two things_:

1. Embeddings, for this we will use `VoyageEmbeddings` using Voyage AI's embedding models, which do need an API-Key
2. A vector database, where we store our embeddings and query them. We use Pinecone, which also needs an API-Key.


In [4]:
import os
from getpass import getpass
from langchain_voyageai import VoyageAIEmbeddings
from pinecone import Pinecone
from pinecone import ServerlessSpec

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or getpass("Pinecone API key: ")
voyage_key = os.getenv("VOYAGE_API_KEY") or getpass("Voyage API key: ")

embed = VoyageAIEmbeddings(
    voyage_api_key=voyage_key, model="voyage-large-2-instruct"
)

# configure client
pc = Pinecone(api_key=api_key)

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

vec = embed.embed_documents(["ello"])
len(vec[0])

batch size None


1024

In [5]:
import time

index_name = "werewolf-index"

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=len(vec[0]),  # dimensionality of voyage model
#        metric='dotproduct',
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 8}},
 'total_vector_count': 8}

In [6]:
import pandas as pd

# create dataframe from knowledge list
df = pd.DataFrame(knowledge)
df

,title,content
0,0. Preparation,## 0. Preparation\n\n#### Precondition\nThe re...
1,3. Game end,## 3. Game end\n\n#### Precondition\nA night i...
2,1. First Night,## 1. First Night\n\n#### Precondition\nIt is ...
3,Rules::The Roles,## Rules::The Roles\n- First things first: All...
4,Rules::Game Situation,## Rules::Game Situation\nWe find ourselves in...
5,4. Day,## 4. Day\n\n#### Precondition\nA night is ove...
6,Rules::Game Objective,## Rules::Game Objective\n- For the villagers:...
7,2. Further Nights,## 2. Further Nights\n\n#### Precondition\nAny...


In [7]:
# embed content
embeddings = embed.embed_documents(df.content.tolist())

# add to Pinecone index
index.upsert(vectors=zip(df.index.astype(str), embeddings, knowledge))

{'upserted_count': 8}

In [8]:
from langchain.agents import tool

@tool
def knowledge_search(query: str) -> str:
#    """Use this tool to augment your query with local knowledge.
    """Provides Knowledge about the game 'Werewolf' based on the query.
    """
    # create query vector
    xq = embed.embed_query(query)
    # perform search
    out = index.query(vector=xq, top_k=2, include_metadata=True)
    # reformat results into string
    results_str = "\n\n".join(
        [x['metadata']['content'] for x in out["matches"]]
    )
    return results_str

tools = [knowledge_search]

In [9]:
from langchain_core.prompts import ChatPromptTemplate

template_text = """
Du bist Geschichtenerzähler und spielst mit einer Runde Kinder im Alter von 10-15 Jahren eine Runde des Spieles "Werwolf".
Die Kinder reichen dir in jeder Runde einen Ausschnitt aus den Regeln, und bitten Dich, die nächste Aktion zu bestimmen.
Bitte unterbreche Deine Erzählung jedesmal, wenn eines der Kinder eine Eingabe machen soll.
Setze dann die Erzählung fort, wenn das Kind die Eingabe gemacht hat.

Du hast Zugang zu folgenden Tools:

{tools}

Um ein Tool zu verwenden, kannst du die Tags <tool></tool> und <tool_input></tool_input> benutzen. 
   Du erhältst dann eine Antwort in der Form <observation></observation>.
Wenn du zum Beispiel ein Tool namens 'search' hast, das eine Google-Suche durchführen kann, 
würdest du für die Suche nach dem Wetter in Frankfurt Folgendes eingeben:

<tool>search</tool><tool_input>Wetter in Frankfurt</tool_input>
<observation>18 Grad</observation>

Wenn du fertig bist, antworte mit einer finalen Antwort zwischen <final_answer></final_answer>. Zum Beispiel:

<final_answer>Das Wetter in Frankfurt beträgt 18 Grad</final_answer>

Los gehts!

Bisheriges Gesprächsprotokoll:
{chat_history}

Frage: {input}
{agent_scratchpad}"""

template_text = """
You are a helpful assistant. Help the user answer any questions.

You have access to the following tools:

{tools}

In order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>
For example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:

<tool>search</tool><tool_input>weather in SF</tool_input>
<observation>64 degrees</observation>

When you are done, respond with a final answer between <final_answer></final_answer>. For example:

<final_answer>The weather in SF is 64 degrees</final_answer>

Begin!

Previous Conversation:
{chat_history}

Question: {input}
{agent_scratchpad}
"""

prompt = ChatPromptTemplate.from_messages([
  ("human", template_text),
])

from pprint import pp

pp(prompt)

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], template="\nYou are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\nWhen you are done, respond with a final answer between <final_answer></final_answer>. For example:\n\n<final_answer>The weather in SF is 64 degrees</final_answer>\n\nBegin!\n\nPrevious Conversation:\n{chat_history}\n\nQuestion: {input}\n{ag

In [11]:
from langchain_anthropic import ChatAnthropic

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY") or getpass("Anthropic API key: ")

# chat completion llm
llm = ChatAnthropic(
    anthropic_api_key=anthropic_api_key,
    model_name="claude-3-opus-20240229",  # change "opus" -> "sonnet" for speed
    temperature=0.0
)

In [12]:
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log


def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])




In [13]:
from langchain.agents.output_parsers import XMLAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        # without "chat_history", tool usage has no context of prev interactions
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tools))
    | llm.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

In [15]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=15,
    return_messages=True
)

conversational_memory.chat_memory.messages

[]

In [16]:
from langchain_core.messages.human import HumanMessage

def memory2str(memory: ConversationBufferWindowMemory):
    messages = memory.chat_memory.messages
    memory_list = [
        f"Human: {mem.content}" if isinstance(mem, HumanMessage) \
        else f"AI: {mem.content}" for mem in messages
    ]
    memory_str = "\n".join(memory_list)
    return memory_str

In [17]:
def chat(text: str):
    out = agent_executor.invoke({
        "input": text,
        "chat_history": memory2str(conversational_memory)
    })
    conversational_memory.chat_memory.add_user_message(text)
    conversational_memory.chat_memory.add_ai_message(out["output"])
    return out["output"]

In [18]:
print(chat("Lass uns eine Runde Werwolf spielen!"))



> Entering new AgentExecutor chain...
Okay, lass uns eine Runde Werwölfe spielen! Hier ist eine kurze Zusammenfassung der Regeln:

<tool>knowledge_search</tool>
<tool_input>Wie spielt man das Spiel Werwölfe?## Rules::Game Situation
We find ourselves in the picturesque village of Düsterwald. But the idyll is deceptive. For some time now,
a pack of werewolves has been up to no good, and every night a
villager falls victim to their insatiable hunger. In an effort to eradicate the evil, the villagers
resort to self-help, and the once tranquil place becomes the stage for a bitterly fought
battle for naked existence.




## Rules::Game Objective
- For the villagers: Eradicate the werewolves and restore peace.
- For the werewolves: Devour or somehow eliminate all the villagers. (Some special roles may have different game objectives)


Okay, hier ist eine kurze Zusammenfassung der wichtigsten Regeln für eine Runde Werwölfe:

<final_answer>

Spielziel: 
- Für die Dorfbewohner: Die Werwölfe au

In [19]:
print(chat("Okay, lass uns zu acht spielen. Du bist Spielleiter. Ich bin Hexe. Die anderen 7 Spieler musst Du simulieren, sag mir bloss nicht welche Rollen Du ihnen zuweist! Das musst Du geheim halten."))



> Entering new AgentExecutor chain...
Okay, dann lass uns loslegen! Ich werde die Rollen für die anderen 7 Spieler geheim auswählen und zuweisen. Du bist die Hexe.

<tool>knowledge_search</tool>
<tool_input>Welche Rollen gibt es typischerweise in einem 8-Personen Werwolf Spiel neben der Hexe?## Rules::The Roles
- First things first: All players who are not werewolves are considered good citizens.
- However, the citizens can also take on special roles:
  - The Seer: Every night, she may inquire about the identity of a fellow player.
  - The Girl: She may blink while the wolves are at work at night.
  - The Witch: She has two potions at her disposal, a healing potion and a poison potion. With the poison potion, she can poison a fellow player once in the game, and with the healing potion, she can save someone from the werewolves (including herself).
  - The Hunter: Should he be killed, he can fire a final shot and drag a fellow player to their doom.
  - Amor: At the beginning of the gam

In [20]:
print(chat("Ich habe eine Augen geschlossen!"))



> Entering new AgentExecutor chain...
Okay, die erste Nacht beginnt.

Werwölfe, wacht auf und einigt euch, wen ihr fressen wollt. Zeigt unauffällig auf euer Opfer.

<tool>knowledge_search</tool>
<tool_input>Was macht die Hexe, nachdem die Werwölfe ihr Opfer gewählt haben?## 2. Further Nights

#### Precondition
Any further after the first round begins. The game master says "The night begins. Everybody close your eyes."

#### Action
- The game master asks the werewolves to open their eyes. The werewolves then decide on a person to kill. The game master tells the werewolves to close their eyes again.
- The game master asks the witch to open their eyes. The game master then shows the witch the person that the werewolves have chosen to kill. The witch then decides whether to save that person or not. The witch also decides whether to kill another person. The game master tells the witch to close their eyes again.
- The game master then says "So the night is over. Everybody open your eyes."


- Agentic RAG Colab : https://colab.research.google.com/drive/1_8r0Tj-f2UyRZ4SBuaP-RfhdUeXXxu0Z?usp=sharing#scrollTo=b8CjRlbVmRpW
- low-code builder: https://flowiseai.com/
- stuff: https://github.com/AI-Maker-Space?q=&type=all&language=&sort=stargazers 
- video: https://www.youtube.com/watch?v=SEA3eJrDc-k